In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.model_selection import KFold
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

2025-02-01 22:55:51.854290: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
def create_model(num_classes, input_shape=(299, 299, 3), dropout_rates=[0.3, 0.4, 0.3]):
    """Create an Xception-based transfer learning model with custom top layers"""
    base_model = applications.Xception(
        weights="imagenet", 
        include_top=False, 
        input_shape=input_shape
    )
    
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    input_layer = Input(shape=input_shape, name='image_input')
    x = base_model(input_layer)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rates[0])(x)
    
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rates[1])(x)
    
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rates[2])(x)
    
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=optimizers.Adam(learning_rate=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def plot_training_history(history, fold=None):
    """Plot training and validation accuracy/loss"""
    plt.figure(figsize=(16, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    
    filename = f'training_history_fold_{fold}.png' if fold is not None else 'training_history.png'
    plt.savefig(filename)
    plt.close()

In [ ]:
def cross_validate_model(train_dir, train_csv, output_dir, n_folds=5, epochs=50):
    """Perform k-fold cross-validation"""

    traindf = pd.read_csv(train_csv)
    traindf['code'] = traindf['type'].astype('category').cat.codes
    
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(traindf), 1):
        print(f'\nTraining Fold {fold}/{n_folds}')
        
        train_df = traindf.iloc[train_idx]
        val_df = traindf.iloc[val_idx]
        
        train_datagen = ImageDataGenerator(
            rescale=1./255.,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        val_datagen = ImageDataGenerator(rescale=1./255.)

        train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            directory=train_dir,
            x_col="chart",
            y_col="type",
            batch_size=32,
            seed=42,
            shuffle=True,
            class_mode="categorical",
            target_size=(299, 299)
        )

        val_generator = val_datagen.flow_from_dataframe(
            dataframe=val_df,
            directory=train_dir,
            x_col="chart",
            y_col="type",
            batch_size=32,
            seed=42,
            shuffle=False,
            class_mode="categorical",
            target_size=(299, 299)
        )

        num_classes = len(traindf['type'].unique())
        model = create_model(num_classes)
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
            ModelCheckpoint(
                os.path.join(output_dir, f'best_model_fold_{fold}.h5'),
                save_best_only=True,
                monitor='val_accuracy'
            )
        ]

        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=epochs,
            callbacks=callbacks,
            verbose=1
        )

        plot_training_history(history, fold)

        # Evaluate
        val_loss, val_accuracy = model.evaluate(val_generator)
        fold_scores.append({
            'fold': fold,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        })
        
        model_json = model.to_json()
        with open(os.path.join(output_dir, f"model_fold_{fold}.json"), "w") as json_file:
            json_file.write(model_json)
        model.save_weights(os.path.join(output_dir, f"model_fold_{fold}.h5"))

    cv_results = pd.DataFrame(fold_scores)
    print("\nCross-validation results:")
    print(cv_results)
    print(f"\nMean validation accuracy: {cv_results['val_accuracy'].mean():.4f}")
    print(f"Standard deviation: {cv_results['val_accuracy'].std():.4f}")
    
    cv_results.to_csv(os.path.join(output_dir, 'cross_validation_results.csv'), index=False)
    return cv_results

In [ ]:
def predict_and_visualize(model, test_dir, test_csv, train_generator, output_dir):
    """Make predictions on test data and visualize results"""
   
    testdf = pd.read_csv(test_csv)
    
    test_datagen = ImageDataGenerator(rescale=1./255.)
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=testdf,
        directory=test_dir,
        x_col="chart",
        y_col=None,
        batch_size=1,
        seed=42,
        shuffle=False,
        class_mode=None,
        target_size=(299, 299)
    )

    test_generator.reset()
    pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
    
    predicted_class_indices = np.argmax(pred, axis=1)
    labels = dict((v,k) for k,v in train_generator.class_indices.items())
    predictions = [labels[k] for k in predicted_class_indices]
    
    results = pd.DataFrame({
        "chart": test_generator.filenames,
        "predicted_type": predictions,
        "confidence": np.max(pred, axis=1)
    })
    results.to_csv(os.path.join(output_dir, "prediction.csv"), index=False)

    barplots = results[results['predicted_type'] == 'BarPlot']
    num_images = min(len(barplots), 100)
    
    if num_images > 0:
        rows = int(np.ceil(np.sqrt(num_images)))
        cols = int(np.ceil(num_images / rows))
        
        fig, axes = plt.subplots(rows, cols, figsize=(20, 20))
        fig.suptitle('Predicted Bar Plot Samples', fontsize=16)
        
        for index in range(num_images):
            image_path = os.path.join(test_dir, barplots.iloc[index]['chart'])
            row = index // cols
            col = index % cols
            
            try:
                img = mpimg.imread(image_path)
                if rows == 1:
                    axes[col].imshow(img)
                    axes[col].axis('off')
                else:
                    axes[row, col].imshow(img)
                    axes[row, col].axis('off')
            except Exception as e:
                print(f"Error loading image {image_path}: {str(e)}")
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'bar_plot_predictions.png'))
        plt.close()

    print("Prediction visualization complete.")
    return results

In [ ]:
if __name__ == "__main__":
   
    TRAIN_DIR = "/root/autodl-tmp/Dataset/Train"
    TRAIN_CSV = "/root/autodl-tmp/Dataset/train.csv"
    TEST_DIR = "/root/autodl-tmp/Dataset/Test"
    TEST_CSV = "/root/autodl-tmp/Dataset/test.csv"
    OUTPUT_DIR = "/root/autodl-tmp/Dataset/model_output"
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    cv_results = cross_validate_model(TRAIN_DIR, TRAIN_CSV, OUTPUT_DIR, n_folds=5, epochs=50)
    
    traindf = pd.read_csv(TRAIN_CSV)
    num_classes = len(traindf['type'].unique())
    final_model = create_model(num_classes)
    
    train_datagen = ImageDataGenerator(
        rescale=1./255.,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=traindf,
        directory=TRAIN_DIR,
        x_col="chart",
        y_col="type",
        batch_size=32,
        seed=42,
        shuffle=True,
        class_mode="categorical",
        target_size=(299, 299)
    )
    
    final_model.fit(
        train_generator,
        epochs=int(cv_results['val_accuracy'].mean() * 50),  # Adjust epochs based on CV performance
        callbacks=[
            EarlyStopping(monitor='loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=1e-6),
            ModelCheckpoint(
                os.path.join(OUTPUT_DIR, 'model.h5'),
                save_best_only=True,
                monitor='loss'
            )
        ],
        verbose=1
    )
    
    model_json = final_model.to_json()
    with open(os.path.join(OUTPUT_DIR, "model.json"), "w") as json_file:
        json_file.write(model_json)
    final_model.save_weights(os.path.join(OUTPUT_DIR, "model.h5"))
    
    predictions = predict_and_visualize(
        final_model,
        TEST_DIR,
        TEST_CSV,
        train_generator,
        OUTPUT_DIR
    )


Training Fold 1/5
Found 16197 validated image filenames belonging to 19 classes.
Found 4050 validated image filenames belonging to 19 classes.


2025-02-01 22:55:53.720920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-01 22:55:54.192159: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:53:00.0, compute capability: 8.6


Epoch 1/50


2025-02-01 22:55:59.793886: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2025-02-01 22:56:01.404794: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


 10/507 [..............................] - ETA: 3:43 - loss: 3.3814 - accuracy: 0.1500

...

507/507 [==============================] - 272s 532ms/step - loss: 0.6167 - accuracy: 0.8461 - val_loss: 0.2615 - val_accuracy: 0.9316 - lr: 1.0000e-04
Epoch 2/50
507/507 [==============================] - 266s 525ms/step - loss: 0.2694 - accuracy: 0.9241 - val_loss: 0.1970 - val_accuracy: 0.9439 - lr: 1.0000e-04
Epoch 3/50
507/507 [==============================] - 269s 530ms/step - loss: 0.2077 - accuracy: 0.9383 - val_loss: 0.1753 - val_accuracy: 0.9474 - lr: 1.0000e-04
Epoch 4/50
507/507 [==============================] - 269s 530ms/step - loss: 0.1717 - accuracy: 0.9479 - val_loss: 0.1676 - val_accuracy: 0.9553 - lr: 1.0000e-04
Epoch 5/50
507/507 [==============================] - 269s 530ms/step - loss: 0.1461 - accuracy: 0.9559 - val_loss: 0.1645 - val_accuracy: 0.9595 - lr: 1.0000e-04
Epoch 6/50
507/507 [==============================] - 268s 529ms/step - loss: 0.1241 - accuracy: 0.9602 - val_loss: 0.1707 - val_accuracy: 0.9531 - lr: 1.0000e-04
Epoch 7/50
507/507 [=============

/root/miniconda3/lib/python3.8/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


633/633 [==============================] - 315s 493ms/step - loss: 0.5331 - accuracy: 0.8645 - lr: 1.0000e-04
Epoch 2/47
633/633 [==============================] - 317s 500ms/step - loss: 0.2518 - accuracy: 0.9299 - lr: 1.0000e-04
Epoch 3/47
633/633 [==============================] - 316s 499ms/step - loss: 0.1986 - accuracy: 0.9409 - lr: 1.0000e-04
Epoch 4/47
633/633 [==============================] - 316s 499ms/step - loss: 0.1598 - accuracy: 0.9505 - lr: 1.0000e-04
Epoch 5/47
633/633 [==============================] - 313s 494ms/step - loss: 0.1353 - accuracy: 0.9564 - lr: 1.0000e-04
Epoch 6/47
633/633 [==============================] - 314s 497ms/step - loss: 0.1195 - accuracy: 0.9604 - lr: 1.0000e-04
Epoch 7/47
633/633 [==============================] - 315s 497ms/step - loss: 0.1058 - accuracy: 0.9653 - lr: 1.0000e-04
Epoch 8/47
633/633 [==============================] - 314s 495ms/step - loss: 0.0981 - accuracy: 0.9671 - lr: 1.0000e-04
Epoch 9/47
633/633 [=======================

/root/miniconda3/lib/python3.8/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


140/140 [==============================] - 2s 9ms/step
Prediction visualization complete.
